<a href="https://colab.research.google.com/github/Hubert26/suicides_IPPAN/blob/main/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
import os
from google.colab import files

#Ustawienie braku maksymalnej ilości wyświetlanych kolumn
pd.options.display.max_columns = None

In [ ]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/Hubert26/suicides_IPPAN/main/data/final_samobojstwa_2013_2022.csv', delimiter=',', low_memory=False)

In [ ]:
df_raw.head()

,ID samobójcy,Data raportu [RRRRMM],Przedział wiekowy,Płeć,Stan cywilny,Wykształcenie,Informacje o pracy i nauce,Źródło utrzymania,Czy samobójstwo zakończyło się zgonem,Miejsce zamachu,Sposób popełnienia,Powód zamachu,Powód zamachu 2,Powód zamachu 3,Powód zamachu 4,Stan świadomości,Informacje o używaniu substancji,Informacje dotyczące leczenia z powodu alkoholizmu/narkomanii
0,"114581552,00",201301,0,2,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0
1,"114584284,00",201301,2,2,0,0,1,0,1,1,1,0,NaN,NaN,NaN,0,0,0
2,"114584293,00",201301,2,1,1,2,1,1,1,15,2,0,NaN,NaN,NaN,0,0,0
3,"114584312,00",201301,3,2,1,0,0,0,1,0,18,1,NaN,NaN,NaN,0,0,0
4,"114584399,00",201301,7,2,1,0,9,0,1,2,3,0,NaN,NaN,NaN,2,0,0


#Zamiana nazw kolumn

In [ ]:
# Zamiana spacji na podkreślenia w nazwach kolumn
df_raw.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)


In [ ]:
df_raw.rename(columns={'Data_raportu_[RRRRMM]': 'Data_raportu'}, inplace=True)

# Daty

In [ ]:
# Wyodrębnienie roku i miesiąca
df_raw['Data_raportu_rok'] = df_raw['Data_raportu'].apply(lambda x: x[0:4] if len(x) > 8 else x[:4])
df_raw['Data_raportu_miesiąc'] = df_raw['Data_raportu'].apply(lambda x: x[5:7] if len(x) > 8 else x[4:6])

In [ ]:
# Połączenie roku i miesiąca w formacie MM.YYYY
df_raw['Data_raportu'] = df_raw['Data_raportu_miesiąc'] + '.' + df_raw['Data_raportu_rok']

#Mapowanie

##Przedział_wiekowy

In [ ]:
column = 'Przedział_wiekowy'
mapping = {
    0: np.nan,
    1: '7-12',
    2: '13-18',
    3: '19-24',
    4: '25-29',
    5: '30-34',
    6: '35-39',
    7: '40-44',
    8: '45-49',
    9: '50-54',
    10: '55-59',
    11: '60-64',
    12: '65-69',
    13: '70-74',
    14: '75-79',
    15: '80-84',
    16: '85+'
}

df_raw[column] = df_raw[column].map(mapping)

##Płeć

In [ ]:
column = 'Płeć'
mapping = {
    0: np.nan,
    1: 'Kobieta',
    2: 'Mężczyzna'
}

df_raw[column] = df_raw[column].map(mapping)


##Stan_cywilny

In [ ]:
column = 'Stan_cywilny'
mapping = {
    0: np.nan,
    1: 'Kawaler/panna',
    2: 'Konkubent/konkubina',
    3: 'Żonaty/zamężna',
    4: 'Separowany/separowana',
    5: 'Rozwiedziony/rozwiedziona',
    6: 'Wdowiec/wdowa'
}

df_raw[column] = df_raw[column].map(mapping)



##Wykształcenie

In [ ]:
column = 'Wykształcenie'
mapping = {
    0: np.nan,  # Zamiana 0 na NaN
    1: 'Podstawowe niepełne',
    2: 'Podstawowe',
    3: 'Gimnazjalne',
    4: 'Zasadnicze zawodowe',
    5: 'Średnie',
    6: 'Policealne',
    7: 'Wyższe'
}

df_raw[column] = df_raw[column].map(mapping)


##Informacje_o_pracy_i_nauce

In [ ]:
column = 'Informacje_o_pracy_i_nauce'
mapping = {
    0: np.nan,  # Zamiana 0 na NaN
    1: 'Uczeń',
    2: 'Student',
    3: 'Pracownik umysłowy',
    4: 'Pracownik fizyczny',
    5: 'Rolnik',
    6: 'Pracujący na własny rachunek/samodzielna działalność gospodarcza',
    7: 'Praca stała',
    8: 'Praca dorywcza',
    9: 'Bezrobotny'
}

df_raw[column] = df_raw[column].map(mapping)

##Źródło_utrzymania

In [ ]:
column = 'Źródło_utrzymania'
mapping = {
    0: np.nan,  # Zamiana 0 na NaN
    1: 'Na utrzymaniu innej osoby',
    2: 'Praca',
    3: 'Renta, emerytura, zasiłek lub alimenty',
    4: 'Nie ma stałego źródła utrzymania'
}

df_raw[column] = df_raw[column].map(mapping)

##Czy_samobójstwo_zakończyło_się_zgonem

In [ ]:
column = 'Czy_samobójstwo_zakończyło_się_zgonem'

mapping = {
    0: np.nan,  # Zamiana 0 na NaN
    1: 1,
    2: 0
}
df_raw[column] = df_raw[column].map(mapping)

##Miejsce_zamachu

In [ ]:
column = 'Miejsce_zamachu'
mapping = {
    0: np.nan,
    1: 'Droga/ulica/chodnik',
    2: 'Zabudowania gospodarcze',
    3: 'Mieszkanie/dom',
    4: 'Teren kolejowy/tory',
    5: 'Park/las',
    6: 'Piwnica, strych, garaż korytarz',
    7: 'Rzeka, jezioro, inny zbiornik wodny',
    8: 'Zakład pracy',
    9: 'Placówka lecznicza lub sanatoryjna',
    10: 'Miejsce prawnej izolacji',
    11: 'Obiekt/jednostka wojskowa',
    12: 'Placówka wychowawczo-opiekuńcza',
    13: 'Szkoła/uczelnia',
    14: 'Obiekt policyjny',
    15: 'Inne'
}

df_raw[column] = df_raw[column].map(mapping)


##Sposób_popełnienia

In [ ]:
column = 'Sposób_popełnienia'
mapping = {
    0: np.nan,
    1: 'Rzucenie się pod pojazd w ruchu',
    2: 'Rzucenie się z wysokości',
    3: 'Powieszenie się',
    4: 'Uszkodzenie układu krwionośnego',
    5: 'Zastrzelenie się/użycie broni palnej',
    6: 'Samookaleczenie powierzchowne',
    7: 'Samookaleczenie inne',
    8: 'Zażycie środków nasennych/leków psychotropowych',
    9: 'Zażycie trucizny',
    10: 'Zatrucie gazem/spalinami',
    11: 'Zażycie innych leków',
    12: 'Zatrucie środkami chemicznymi/toksycznymi',
    13: 'Zatrucie środkami odurzającymi',
    14: 'Zatrucie dopalaczami',
    15: 'Utonięcie/utopienie się',
    16: 'Samopodpalenie',
    17: 'Uduszenie',
    18: 'Inny'
}

df_raw[column] = df_raw[column].map(mapping)


##Powód_zamachu

In [ ]:
column = 'Powód_zamachu'
mapping = {
    0: np.nan,
    1: 'Zawód miłosny',
    2: 'Choroba psychiczna/zaburzenia psychiczne',
    3: 'Nieporozumienie rodzinne/przemoc w rodzinie',
    4: 'Nosiciel wirusa HIV, chory na AIDS',
    5: 'Nagła utrata źródła utrzymania',
    6: 'Złe warunki ekonomiczne/długi',
    7: 'Przewlekła choroba',
    8: 'Problemy w szkole lub pracy',
    9: 'Śmierć bliskiej osoby',
    10: 'Dokonanie przestępstwa lub wykroczenia',
    11: 'Trwałe kalectwo',
    12: 'Niepożądana ciąża',
    13: 'Choroba fizyczna',
    14: 'Pogorszenie lub nagła utrata zdrowia',
    15: 'Konflikt z osobami spoza rodziny',
    16: 'Zagrożenie lub utrata miejsca zamieszkania',
    17: 'Mobbing, cybermobbing, znęcanie',
    18: 'Inny niewymieniony'
}

df_raw[column] = df_raw[column].map(mapping)

##Powód_zamachu_2

In [ ]:
column = 'Powód_zamachu_2'
mapping = {
    0: np.nan,
    1: 'Zawód miłosny',
    2: 'Choroba psychiczna/zaburzenia psychiczne',
    3: 'Nieporozumienie rodzinne/przemoc w rodzinie',
    4: 'Nosiciel wirusa HIV, chory na AIDS',
    5: 'Nagła utrata źródła utrzymania',
    6: 'Złe warunki ekonomiczne/długi',
    7: 'Przewlekła choroba',
    8: 'Problemy w szkole lub pracy',
    9: 'Śmierć bliskiej osoby',
    10: 'Dokonanie przestępstwa lub wykroczenia',
    11: 'Trwałe kalectwo',
    12: 'Niepożądana ciąża',
    13: 'Choroba fizyczna',
    14: 'Pogorszenie lub nagła utrata zdrowia',
    15: 'Konflikt z osobami spoza rodziny',
    16: 'Zagrożenie lub utrata miejsca zamieszkania',
    17: 'Mobbing, cybermobbing, znęcanie',
    18: 'Inny niewymieniony'
}

df_raw[column] = df_raw[column].map(mapping)

##Powód_zamachu_3

In [ ]:
column = 'Powód_zamachu_3'
mapping = {
    0: np.nan,
    1: 'Zawód miłosny',
    2: 'Choroba psychiczna/zaburzenia psychiczne',
    3: 'Nieporozumienie rodzinne/przemoc w rodzinie',
    4: 'Nosiciel wirusa HIV, chory na AIDS',
    5: 'Nagła utrata źródła utrzymania',
    6: 'Złe warunki ekonomiczne/długi',
    7: 'Przewlekła choroba',
    8: 'Problemy w szkole lub pracy',
    9: 'Śmierć bliskiej osoby',
    10: 'Dokonanie przestępstwa lub wykroczenia',
    11: 'Trwałe kalectwo',
    12: 'Niepożądana ciąża',
    13: 'Choroba fizyczna',
    14: 'Pogorszenie lub nagła utrata zdrowia',
    15: 'Konflikt z osobami spoza rodziny',
    16: 'Zagrożenie lub utrata miejsca zamieszkania',
    17: 'Mobbing, cybermobbing, znęcanie',
    18: 'Inny niewymieniony'
}

df_raw[column] = df_raw[column].map(mapping)

##Powód_zamachu_4

In [ ]:
column = 'Powód_zamachu_4'
mapping = {
    0: np.nan,
    1: 'Zawód miłosny',
    2: 'Choroba psychiczna/zaburzenia psychiczne',
    3: 'Nieporozumienie rodzinne/przemoc w rodzinie',
    4: 'Nosiciel wirusa HIV, chory na AIDS',
    5: 'Nagła utrata źródła utrzymania',
    6: 'Złe warunki ekonomiczne/długi',
    7: 'Przewlekła choroba',
    8: 'Problemy w szkole lub pracy',
    9: 'Śmierć bliskiej osoby',
    10: 'Dokonanie przestępstwa lub wykroczenia',
    11: 'Trwałe kalectwo',
    12: 'Niepożądana ciąża',
    13: 'Choroba fizyczna',
    14: 'Pogorszenie lub nagła utrata zdrowia',
    15: 'Konflikt z osobami spoza rodziny',
    16: 'Zagrożenie lub utrata miejsca zamieszkania',
    17: 'Mobbing, cybermobbing, znęcanie',
    18: 'Inny niewymieniony'
}

df_raw[column] = df_raw[column].map(mapping)

##Stan_świadomości

In [ ]:
column = 'Stan_świadomości'
mapping = {
    0: np.nan,
    1: 'Trzeźwy',
    2: 'Pod wpływem alkoholu',
    3: 'Pod wpływem zastępczych środków/substancji (dopalaczy)',
    4: 'Pod wpływem substancji psychotropowych (narkotyki, itp.)',
    5: 'Pod wpływem leków',
    6: 'Pod wpływem środków odurzających',
    7: 'Pod wpływem alkoholu i zastępczych środków/substancji (dopalaczy)',
    8: 'Pod wpływem alkoholu i substancji psychotropowych (narkotyki, itp.)',
    9: 'Pod wpływem alkoholu i leków',
    10: 'Pod wpływem alkoholu i środków odurzających',
    11: 'Pod wpływem alkoholu, zastępczych środków/substancji (dopalaczy) i środków odurzających',
    12: 'Pod wpływem zastępczych środków/substancji (dopalaczy) i środków odurzających',
    13: 'Pod wpływem leków i środków odurzających',
    14: 'Pod wpływem leków i zastępczych środków/substancji (dopalaczy)',
    15: 'Pod wpływem leków i substancji psychotropowych (narkotyki, itp.)',
    16: 'Pod wpływem leków, zastępczych środków/substancji (dopalaczy) i środków odurzających',
    17: 'Pod wpływem leków, alkoholu i środków odurzających',
    18: 'Pod wpływem alkoholu, leków, zastępczych środków/substancji (dopalaczy) i środków odurzających'
}

df_raw[column] = df_raw[column].map(mapping)


##Informacje_o_używaniu_substancji

In [ ]:
column = 'Informacje_o_używaniu_substancji'
mapping = {
    0: np.nan,
    1: 'Nadużywał alkoholu',
    2: 'Nadużywał narkotyków',
    3: 'Nadużywał dopalaczy',
    4: 'Nadużywał alkoholu i dopalaczy',
    5: 'Nadużywał narkotyków i dopalaczy',
    6: 'Nadużywał alkoholu i narkotyków',
    7: 'Nadużywał alkoholu, narkotyków i dopalaczy'
}

df_raw[column] = df_raw[column].map(mapping)


##Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii

In [ ]:
column = 'Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii'
mapping = {
    0: np.nan,
    1: 'Leczony z powodu alkoholizmu',
    2: 'Leczony z powodu narkomanii',
    3: 'Leczony z powodu alkoholizmu i narkomanii'
}

df_raw[column] = df_raw[column].map(mapping)


#Powód zamachu - dummies

In [ ]:
columns_to_drop = ['Powód_zamachu', 'Powód_zamachu_2', 'Powód_zamachu_3', 'Powód_zamachu_4']
df_powód_zamachu = df_raw[columns_to_drop]
df_raw.drop(columns=columns_to_drop, inplace=True)


In [ ]:
powód_zamachu_values = (
    set(df_powód_zamachu['Powód_zamachu'].unique()) |
    set(df_powód_zamachu['Powód_zamachu_2'].unique()) |
    set(df_powód_zamachu['Powód_zamachu_3'].unique()) |
    set(df_powód_zamachu['Powód_zamachu_4'].unique())
)

In [ ]:
powód_zamachu_values

{'Choroba fizyczna',
 'Choroba psychiczna/zaburzenia psychiczne',
 'Dokonanie przestępstwa lub wykroczenia',
 'Inny niewymieniony',
 'Konflikt z osobami spoza rodziny',
 'Mobbing, cybermobbing, znęcanie',
 'Nagła utrata źródła utrzymania',
 'Nieporozumienie rodzinne/przemoc w rodzinie',
 'Niepożądana ciąża',
 'Nosiciel wirusa HIV, chory na AIDS',
 'Pogorszenie lub nagła utrata zdrowia',
 'Problemy w szkole lub pracy',
 'Przewlekła choroba',
 'Trwałe kalectwo',
 'Zagrożenie lub utrata miejsca zamieszkania',
 'Zawód miłosny',
 'Złe warunki ekonomiczne/długi',
 nan,
 'Śmierć bliskiej osoby'}

In [ ]:
for value in powód_zamachu_values:
    column_name = 'Powód_zamachu_' + str(value)
    df_powód_zamachu[column_name] = df_powód_zamachu.apply(lambda row: 1 if value in row.values else 0, axis=1)


<ipython-input-29-f9e5e51963ac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_powód_zamachu[column_name] = df_powód_zamachu.apply(lambda row: 1 if value in row.values else 0, axis=1)
<ipython-input-29-f9e5e51963ac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_powód_zamachu[column_name] = df_powód_zamachu.apply(lambda row: 1 if value in row.values else 0, axis=1)
<ipython-input-29-f9e5e51963ac>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [ ]:
columns_to_drop = ['Powód_zamachu', 'Powód_zamachu_2', 'Powód_zamachu_3', 'Powód_zamachu_4', 'Powód_zamachu_nan']
df_powód_zamachu.drop(columns=columns_to_drop, inplace=True)


#Liczba_powodów_zamachu

In [ ]:
df_raw['Liczba_powodów_zamachu'] = df_powód_zamachu.sum(axis=1)

In [ ]:
df_raw.head()

,ID_samobójcy,Data_raportu,Przedział_wiekowy,Płeć,Stan_cywilny,Wykształcenie,Informacje_o_pracy_i_nauce,Źródło_utrzymania,Czy_samobójstwo_zakończyło_się_zgonem,Miejsce_zamachu,Sposób_popełnienia,Stan_świadomości,Informacje_o_używaniu_substancji,Informacje_dotyczące_leczenia_z_powodu_alkoholizmu/narkomanii,Data_raportu_rok,Data_raportu_miesiąc,Liczba_powodów_zamachu
0,"114581552,00",01.2013,NaN,Mężczyzna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,01,0
1,"114584284,00",01.2013,NaN,Mężczyzna,NaN,NaN,Uczeń,NaN,1.0,Droga/ulica/chodnik,Rzucenie się pod pojazd w ruchu,NaN,NaN,NaN,2013,01,0
2,"114584293,00",01.2013,NaN,Kobieta,Kawaler/panna,Podstawowe,Uczeń,Na utrzymaniu innej osoby,1.0,Inne,Rzucenie się z wysokości,NaN,NaN,NaN,2013,01,0
3,"114584312,00",01.2013,NaN,Mężczyzna,Kawaler/panna,NaN,NaN,NaN,1.0,NaN,Inny,NaN,NaN,NaN,2013,01,1
4,"114584399,00",01.2013,NaN,Mężczyzna,Kawaler/panna,NaN,Bezrobotny,NaN,1.0,Zabudowania gospodarcze,Powieszenie się,Pod wpływem alkoholu,NaN,NaN,2013,01,0


#Zapis

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# df_raw.to_csv('/content/drive/My Drive/preped_samobojstwa_2013_2022.csv', index=False)
# df_powód_zamachu.to_csv('/content/drive/My Drive/preped_powod_zamachu_samobojstwa_2013_2022.csv', index=False)

In [ ]:
file_name = 'preped_samobojstwa_2013_2022.csv'
df_raw.to_csv(file_name, index=False)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
file_name = 'preped_powod_zamachu_samobojstwa_2013_2022.csv'
df_raw.to_csv(file_name, index=False)
files.download(file_name)